In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00


In [132]:
''' bpe design
   letter table {token: <letter>}
   token table {token: (token1, token2)}

train:
   most frequent token pair
   if pair count >= min_count and token table size < max_tokens
      create new token for the pair
   if new token is not empty
      for each pair in sequence
          replace pair with new token
      add new token to token table
   else
      break

encode:
  convert sequence to token sequence
  do
    for each pair in sequence:
      replace pair with token if found in token table
  until no pairs can be tokenized

decode:
  do
    for each token in sequence
      replace with pair
  until no token can be decoded
'''
import tiktoken
import heapq

enc = tiktoken.get_encoding('gpt2')
len(enc.encode('bpe'))

words = 'do you still remember the day of world david...'
letters = list(sorted(set(words)))

# vocab
a2t = {ch:token for token,ch in enumerate(letters)}
t2a = {token:ch for ch,token in a2t.items()}
t2p= {token:(token,) for token,_ in t2a.items()}
p2t= {}


# train
data = [a2t[ch] for ch in words]
def count_pairs(data):
  counter = dict()
  for i in range(len(data)-1):
    pair = (data[i],data[i+1])
    if pair in counter:
      counter[pair] += 1
    else:
      counter[pair] = 1

  if len(counter) == 0:
    return None
  else:
    return heapq.nlargest(1, counter.items(), lambda x:x[1])[-1]

min_count = 2
max_tokens = 100

# train
def train(data):
  global p2t
  while True:
    pair = count_pairs(data)
    if pair is None or pair[1] < min_count or len(ttot) >= max_tokens:
      break
    else:
      print('make token for ', pair)
      token = len(t2p)
      t2p[token] = pair[0]
      new_data = []
      pos = 0
      while pos < len(data) - 1:
        if data[pos] == pair[0][0] and data[pos+1] == pair[0][1]:
          new_data.append(token)
          pos += 2
        else:
          new_data.append(data[pos])
          pos += 1
      if pos < len(data):
        new_data.append(data[pos])
      data = new_data
    p2t = {pair:token for token,pair in t2p.items()}

# encode
def encode(text):
  ''' given atot and ttot, convert text to token sequence'''
  tokens = [a2t[ch] for ch in text]
  while True:
    new_tokens = []
    pos = 0
    while pos < len(tokens) - 1:
      if (tokens[pos], tokens[pos+1]) in p2t:
        #print('replace', (tokens[pos], tokens[pos+1]), 'with', p2t[(tokens[pos], tokens[pos+1])])
        new_tokens.append(p2t[(tokens[pos], tokens[pos+1])])
        pos += 2
      else:
        new_tokens.append(tokens[pos])
        pos += 1
    if pos < len(tokens):
      new_tokens.append(tokens[pos])
    if len(new_tokens) == len(tokens):
      break
    else:
      tokens = new_tokens

  return tokens

# decode
def decode(tokens):
  while True:
    i = 0
    new_tokens = []
    for token in tokens:
      i += 1
      if i > 100:
        break
      if token in t2p:
        new_tokens.append(t2p[token][0])
        if len(t2p[token]) == 2:
          new_tokens.append(t2p[token][1])
    if len(new_tokens) == len(tokens):
      break
    else:
      tokens = new_tokens
  return ''.join([t2a[token] for token in tokens])

# test
text = words
print('text:', text)
train(data)

print('a2t',a2t)
print('t2a',t2a)
print('t2p',t2p)
print('p2t',p2t)

tokens = encode(text)
print('encode result:',tokens)
print(len(text),'->', len(tokens))
print('decode result:', decode(tokens))


text: do you still remember the day of world david...
make token for  ((5, 10), 2)
make token for  ((0, 4), 2)
make token for  ((20, 2), 2)
make token for  ((1, 1), 2)
a2t {' ': 0, '.': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'f': 6, 'h': 7, 'i': 8, 'l': 9, 'm': 10, 'o': 11, 'r': 12, 's': 13, 't': 14, 'u': 15, 'v': 16, 'w': 17, 'y': 18}
t2a {0: ' ', 1: '.', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'f', 7: 'h', 8: 'i', 9: 'l', 10: 'm', 11: 'o', 12: 'r', 13: 's', 14: 't', 15: 'u', 16: 'v', 17: 'w', 18: 'y'}
t2p {0: (0,), 1: (1,), 2: (2,), 3: (3,), 4: (4,), 5: (5,), 6: (6,), 7: (7,), 8: (8,), 9: (9,), 10: (10,), 11: (11,), 12: (12,), 13: (13,), 14: (14,), 15: (15,), 16: (16,), 17: (17,), 18: (18,), 19: (5, 10), 20: (0, 4), 21: (20, 2), 22: (1, 1)}
p2t {(0,): 0, (1,): 1, (2,): 2, (3,): 3, (4,): 4, (5,): 5, (6,): 6, (7,): 7, (8,): 8, (9,): 9, (10,): 10, (11,): 11, (12,): 12, (13,): 13, (14,): 14, (15,): 15, (16,): 16, (17,): 17, (18,): 18, (5, 10): 19, (0, 4): 20, (20, 2): 21, (1, 1): 22}
encode resul

In [122]:
import heapq

heapq.nlargest(1, [1,3,5,7], lambda x:-x)
i=0
while i < 0:
  print(i)
  i+=1


min_count = 2
def train(data):
  print(data)
  #min_count +=1
  print(min_count)

train(None)

min_count=[]
def bar(data):
  print(min_count, id(min_count))
  min_count.append(3)
  print(min_count, id(min_count))

bar(None)
print(foo)

def test(arr):
  print('arr:',arr)
  arr = [7,8,9]
  for i in arr:
    print('arr:',i)

test([1,2,3])


None
2
[] 134630373861888
[3] 134630373861888
1
arr: [1, 2, 3]
arr: 7
arr: 8
arr: 9
